In [1]:
# for reading and validating data
import emeval.input.spec_details as eisd
import emeval.input.phone_view as eipv
import emeval.input.eval_view as eiev

In [2]:
# Visualization helpers
import emeval.viz.phone_view as ezpv
import emeval.viz.eval_view as ezev

In [3]:
# for pipelined data
import emeval.analysed.phone_view as eapv

In [4]:
import pandas as pd

In [5]:
import geopandas as gpd
import shapely as shp

In [6]:
import numpy as np

In [7]:
import emeval.metrics.dist_calculations as emd

In [8]:
# For plots
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
# Analytics results
import emeval.metrics.segmentation as ems

In [10]:
# For easier debugging while working on modules
import importlib

In [11]:
def import_sd_and_pv_from_server(trips  = ["unimodal_trip_car_bike_mtv_la", "car_scooter_brex_san_jose", "train_bus_ebike_mtv_ucb"], 
                                 AUTHOR_EMAIL  = "shankari@eecs.berkeley.edu", 
                                 DATASTORE_LOC = "http://localhost:8080", 
                                 pkl_file_name = None):
    sd_l = []
    pv_l = []
    for trip in trips:
        sd = eisd.ServerSpecDetails(DATASTORE_LOC, AUTHOR_EMAIL, trip)
        pv = eipv.PhoneView(sd)
        sd_l.append(sd)
        pv_l.append(pv)
    if pkl_file_name:
        import pickle
        with open(pkl_file_name, 'wb') as outp:
            for pv in pv_l:
                pickle.dump(pv, outp, pickle.HIGHEST_PROTOCOL)
    return sd_l, pv_l

In [12]:
def import_pv_from_pkl(pkl_file_name, 
                       trips = ["unimodal_trip_car_bike_mtv_la", "car_scooter_brex_san_jose", "train_bus_ebike_mtv_ucb"]):
    import pickle
    pv_l = []
    with open('pv.pkl', 'rb') as inp:
        for trip in trips:
            pv_l.append(pickle.load(inp))
    return pv_l

In [13]:
(pv_la, pv_sj, pv_ucb) = import_pv_from_pkl('pv.pkl')

In [17]:
av_la = eapv.create_analysed_view(pv_la, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/cleaned_section")

Finished copying unimodal_trip_car_bike_mtv_la, starting overwrite
=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*
android dict_keys(['ucb-sdb-android-1', 'ucb-sdb-android-2', 'ucb-sdb-android-3', 'ucb-sdb-android-4'])
     -*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
     ucb-sdb-android-1 accuracy_control dict_keys(['role', 'manual/evaluation_transition', 'calibration_transitions', 'calibration_ranges', 'evaluation_transitions', 'evaluation_ranges'])
Retrieving data for ucb-sdb-android-1 from 1563606000 -> 1656454719.864118
Retrieving data for: post_body={'user': 'ucb-sdb-android-1', 'key_list': ['analysis/recreated_location'], 'start_time': 1563606000, 'end_time': 1656454719.864118}
response=<Response [200]>
Found 82 entries
Retrieving data for ucb-sdb-android-1 from 1655748331.215638 -> 1656454719.864118
Retrieving data for: post_body={'user': 'ucb-sdb-android-1', 'key_list': ['analysis/recreated_location'], 'start_time': 1655748331.215638, 'end_time': 1656454719.864118}
response=<Response [200]>
Found 1 entries

response=<Response [200]>
Found 11996 entries
Retrieving data for ucb-sdb-android-2 from 1655760626.848227 -> 1656454720.151926
Retrieving data for: post_body={'user': 'ucb-sdb-android-2', 'key_list': ['analysis/recreated_location'], 'start_time': 1655760626.848227, 'end_time': 1656454720.151926}
response=<Response [200]>
Found 1 entries
Retrieving data for ucb-sdb-android-2 from 1563606000 -> 1656454724.824091
Retrieving data for: post_body={'user': 'ucb-sdb-android-2', 'key_list': ['analysis/cleaned_trip'], 'start_time': 1563606000, 'end_time': 1656454724.824091}
response=<Response [200]>
Found 172 entries
Retrieving data for ucb-sdb-android-2 from 1655760622.5240479 -> 1656454724.824091
Retrieving data for: post_body={'user': 'ucb-sdb-android-2', 'key_list': ['analysis/cleaned_trip'], 'start_time': 1655760622.5240479, 'end_time': 1656454724.824091}
response=<Response [200]>
Found 1 entries
Retrieving data for ucb-sdb-android-2 from 1563606000 -> 1656454725.011585
Retrieving data for

Before filtering, trips = [('2019-06-19T14:13:22.042000-07:00', '2019-06-19T14:40:39-07:00'), ('2019-06-21T09:52:45.337000-07:00', '2019-06-21T10:04:41.606000-07:00'), ('2019-06-21T11:50:40-07:00', '2019-06-21T11:55:52-07:00'), ('2019-07-12T10:08:30-07:00', '2019-07-12T10:12:46-07:00'), ('2019-07-12T10:13:17-07:00', '2019-07-12T10:16:28-07:00'), ('2019-07-18T15:55:33-07:00', '2019-07-18T16:04:01-07:00'), ('2019-07-18T16:04:02-07:00', '2019-07-18T16:06:15-07:00'), ('2019-07-18T16:06:16-07:00', '2019-07-18T16:06:40-07:00'), ('2019-07-18T16:06:42-07:00', '2019-07-18T16:14:17-07:00'), ('2019-07-18T16:16:45-07:00', '2019-07-18T16:25:01-07:00'), ('2019-07-18T16:25:02.700000-07:00', '2019-07-18T16:40:23-07:00'), ('2019-07-18T18:06:50.622811-07:00', '2019-07-18T18:26:23-07:00'), ('2019-07-18T18:26:24-07:00', '2019-07-18T18:35:08-07:00'), ('2019-07-18T18:35:09-07:00', '2019-07-18T18:46:12-07:00'), ('2019-07-19T10:52:37-07:00', '2019-07-19T11:05:34-07:00'), ('2019-07-19T11:05:35-07:00', '2019-07

response=<Response [200]>
Found 12795 entries
Retrieving data for ucb-sdb-android-3 from 1655764692.407965 -> 1656454725.598708
Retrieving data for: post_body={'user': 'ucb-sdb-android-3', 'key_list': ['analysis/recreated_location'], 'start_time': 1655764692.407965, 'end_time': 1656454725.598708}
response=<Response [200]>
Found 1 entries
Retrieving data for ucb-sdb-android-3 from 1563606000 -> 1656454729.417229
Retrieving data for: post_body={'user': 'ucb-sdb-android-3', 'key_list': ['analysis/cleaned_trip'], 'start_time': 1563606000, 'end_time': 1656454729.417229}
response=<Response [200]>
Found 174 entries
Retrieving data for ucb-sdb-android-3 from 1655764686.870524 -> 1656454729.417229
Retrieving data for: post_body={'user': 'ucb-sdb-android-3', 'key_list': ['analysis/cleaned_trip'], 'start_time': 1655764686.870524, 'end_time': 1656454729.417229}
response=<Response [200]>
Found 1 entries
Retrieving data for ucb-sdb-android-3 from 1563606000 -> 1656454729.606306
Retrieving data for: 

         dict_keys(['trip_id', 'trip_id_base', 'trip_run', 'start_ts', 'end_ts', 'duration', 'eval_common_trip_id', 'eval_role', 'eval_role_base', 'eval_role_run', 'evaluation_trip_ranges', 'background/battery', 'battery_df', 'background/location', 'background/filtered_location', 'location_df', 'filtered_location_df', 'background/motion_activity', 'motion_activity_df', 'statemachine/transition', 'transition_df'])
         HAHFDC v/s MAHFDC:MAHFDC_2 HAHFDC v/s MAHFDC MAHFDC_2 2
Before filtering, trips = [('2019-06-19T14:13:32-07:00', '2019-06-19T14:41:05-07:00'), ('2019-06-21T11:48:53-07:00', '2019-06-21T11:56:31-07:00'), ('2019-06-21T15:29:54.764000-07:00', '2019-06-21T15:33:59-07:00'), ('2019-06-21T16:13:51.443866-07:00', '2019-06-21T16:20:33-07:00'), ('2019-06-21T16:24:42.645606-07:00', '2019-06-21T18:53:16.074000-07:00'), ('2019-07-12T10:05:51.634187-07:00', '2019-07-12T10:15:04-07:00'), ('2019-07-12T10:18:04.123513-07:00', '2019-07-12T10:20:24-07:00'), ('2019-07-18T16:03:30.668000-

...

         dict_keys(['trip_id', 'trip_id_base', 'trip_run', 'start_ts', 'end_ts', 'duration', 'eval_common_trip_id', 'eval_role', 'eval_role_base', 'eval_role_run', 'evaluation_trip_ranges', 'background/battery', 'battery_df', 'background/location', 'background/filtered_location', 'location_df', 'filtered_location_df', 'background/motion_activity', 'motion_activity_df', 'statemachine/transition', 'transition_df'])
         fixed:ACCURACY_CONTROL_5 HAHFDC v/s HAMFDC accuracy_control_2 2
Before filtering, trips = [('2019-06-21T07:54:37.402644-07:00', '2019-06-21T08:03:12.998210-07:00'), ('2019-06-21T09:54:01.274519-07:00', '2019-06-21T10:07:03.991278-07:00'), ('2019-06-21T10:57:32.626916-07:00', '2019-06-21T11:05:14.993758-07:00'), ('2019-06-21T11:47:37.377343-07:00', '2019-06-21T12:00:44.998291-07:00'), ('2019-06-21T15:27:32.942929-07:00', '2019-06-21T15:34:53.995085-07:00'), ('2019-06-21T16:14:12.935650-07:00', '2019-06-21T16:32:12.144650-07:00'), ('2019-07-12T10:06:29.825268-07:00', '2

         dict_keys(['trip_id', 'trip_id_base', 'trip_run', 'start_ts', 'end_ts', 'duration', 'eval_common_trip_id', 'eval_role', 'eval_role_base', 'eval_role_run', 'evaluation_trip_ranges', 'background/battery', 'battery_df', 'background/location', 'background/filtered_location', 'location_df', 'filtered_location_df', 'background/motion_activity', 'motion_activity_df', 'statemachine/transition', 'transition_df'])
         HAHFDC v/s HAMFDC:HAHFDC_2 HAHFDC v/s HAMFDC HAHFDC_2 2
Before filtering, trips = [('2019-06-19T14:16:16.586914-07:00', '2019-06-19T14:50:51.988175-07:00'), ('2019-06-19T15:03:03.258420-07:00', '2019-06-19T15:14:04.996495-07:00'), ('2019-06-19T16:47:51.154749-07:00', '2019-06-19T17:02:20.989934-07:00'), ('2019-06-19T18:58:04.875024-07:00', '2019-06-19T19:13:21.999279-07:00'), ('2019-06-21T07:51:03.140002-07:00', '2019-06-21T08:04:12.995818-07:00'), ('2019-06-21T11:47:41.075602-07:00', '2019-06-21T12:04:25.995581-07:00'), ('2019-06-21T12:26:29.816319-07:00', '2019-06-2

response=<Response [200]>
Found 11685 entries
Retrieving data for ucb-sdb-ios-3 from 1655765735.6815438 -> 1656454735.125194
Retrieving data for: post_body={'user': 'ucb-sdb-ios-3', 'key_list': ['analysis/recreated_location'], 'start_time': 1655765735.6815438, 'end_time': 1656454735.125194}
response=<Response [200]>
Found 1 entries
Retrieving data for ucb-sdb-ios-3 from 1563606000 -> 1656454739.146898
Retrieving data for: post_body={'user': 'ucb-sdb-ios-3', 'key_list': ['analysis/cleaned_trip'], 'start_time': 1563606000, 'end_time': 1656454739.146898}
response=<Response [200]>
Found 100 entries
Retrieving data for ucb-sdb-ios-3 from 1655765734.756284 -> 1656454739.146898
Retrieving data for: post_body={'user': 'ucb-sdb-ios-3', 'key_list': ['analysis/cleaned_trip'], 'start_time': 1655765734.756284, 'end_time': 1656454739.146898}
response=<Response [200]>
Found 1 entries
Retrieving data for ucb-sdb-ios-3 from 1563606000 -> 1656454739.348599
Retrieving data for: post_body={'user': 'ucb-sd

         dict_keys(['trip_id', 'trip_id_base', 'trip_run', 'start_ts', 'end_ts', 'duration', 'eval_common_trip_id', 'eval_role', 'eval_role_base', 'eval_role_run', 'evaluation_trip_ranges', 'background/battery', 'battery_df', 'background/location', 'background/filtered_location', 'location_df', 'filtered_location_df', 'background/motion_activity', 'motion_activity_df', 'statemachine/transition', 'transition_df'])
         HAHFDC v/s HAMFDC:HAMFDC_1 HAHFDC v/s HAMFDC HAMFDC_1 2
Before filtering, trips = [('2019-06-19T14:16:07.744506-07:00', '2019-06-19T14:42:28.989396-07:00'), ('2019-06-19T15:04:27.431708-07:00', '2019-06-19T15:11:59.984994-07:00'), ('2019-06-19T16:48:51.988024-07:00', '2019-06-19T17:01:27.989144-07:00'), ('2019-06-19T18:57:56.388647-07:00', '2019-06-19T19:12:42.997152-07:00'), ('2019-06-21T07:54:09.506301-07:00', '2019-06-21T08:03:10.996361-07:00'), ('2019-06-21T09:53:57.000253-07:00', '2019-06-21T10:06:55.986031-07:00'), ('2019-06-21T11:45:32.725421-07:00', '2019-06-2

response=<Response [200]>
Found 1 entries
Retrieving data for ucb-sdb-ios-4 from 1563606000 -> 1656454740.159386
Retrieving data for: post_body={'user': 'ucb-sdb-ios-4', 'key_list': ['analysis/cleaned_section'], 'start_time': 1563606000, 'end_time': 1656454740.159386}
response=<Response [200]>
Found 5 entries
Retrieving data for ucb-sdb-ios-4 from 1655765755.889224 -> 1656454740.159386
Retrieving data for: post_body={'user': 'ucb-sdb-ios-4', 'key_list': ['analysis/cleaned_section'], 'start_time': 1655765755.889224, 'end_time': 1656454740.159386}
response=<Response [200]>
Found 1 entries
         dict_keys(['trip_id', 'trip_id_base', 'trip_run', 'start_ts', 'end_ts', 'duration', 'eval_common_trip_id', 'eval_role', 'eval_role_base', 'eval_role_run', 'evaluation_trip_ranges', 'background/battery', 'battery_df', 'background/location', 'background/filtered_location', 'location_df', 'filtered_location_df', 'background/motion_activity', 'motion_activity_df', 'statemachine/transition', 'transi

In [24]:
pv = av_la
for phone_os, phone_map in pv.map().items():
    for phone_label, phone_detail_map in phone_map.items():
        for r in phone_detail_map["evaluation_ranges"]:
            for tr in r["evaluation_trip_ranges"]:
                for ss in tr['sensed_section_ranges']:
                    pass
                for sr in tr['evaluation_section_ranges']:
                    section_gt_leg = pv.spec_details.get_ground_truth_for_leg(tr["trip_id_base"], sr["trip_id_base"], tr['start_ts'], tr['end_ts'])
                    section_gt_shapes = gpd.GeoSeries(eisd.SpecDetails.get_shapes_for_leg(section_gt_leg))
                    section_gt_shapes_df = pd.DataFrame(section_gt_shapes)
                    

/opt/anaconda3/envs/emissioneval/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/opt/anaconda3/envs/emissioneval/lib/python3.8/site-packages/geopandas/geoseries.py:133: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  data = from_shapely(s.values)
/opt/anaconda3/envs/emissioneval/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/opt/anaconda3/envs/emissioneval/lib/python3.8/site-packages/geopandas/geoseries.py:133: ShapelyDeprecationWarning: The array interfac